In [1]:
from tensorflow import keras
import tensorflow as tf
import json
from pathlib import Path
import random
import numpy as np
import librosa
import os
import soundfile as sf
from keras.utils.data_utils import Sequence

In [2]:
import os

In [ ]:
from music_decomp_data_generator import SolosDataGenerator
from unet_implementation.decomp_implimentation import Unet

C:\Users\User\Documents\GitHub\music-decomp\AI\Music Decomposition\music_decomp_data_generator.py:163: ComplexWarning: Casting complex values to real discards the imaginary part
  spectrograms[source_index, audio_index, :, :, 1] = phase


In [ ]:
data_dir_wav = os.path.abspath(r"..\..\Solos-Files\data_files\audio_wav")
data_dir_wav

In [ ]:
dm = SolosDataGenerator(data_dir_wav)

In [11]:
%%time
dm[0]

KeyboardInterrupt: 